<a href="https://colab.research.google.com/github/heena2248/Projects/blob/main/privacy_preserving_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people

# Step 1: Load and Preprocess a Smaller Dataset
print("Loading Labeled Faces in the Wild dataset...")
lfw = fetch_lfw_people(min_faces_per_person=50, resize=0.4)  # Use fewer identities
images, labels = lfw.images, lfw.target
images = images / 255.0  # Normalize images to [0, 1]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)

print(f"Dataset loaded: {len(X_train)} training samples, {len(X_test)} testing samples")

# Step 2: Define Differential Privacy Mechanism
def add_gaussian_noise(tensor, epsilon=2.0):
    """
    Add Gaussian noise to tensor for differential privacy.
    Lower epsilon => higher noise => more privacy.
    """
    noise = tf.random.normal(shape=tf.shape(tensor), stddev=1/epsilon)
    return tensor + noise

# Step 3: Define Partitioned VGG-Face-like Network
def create_client_model(input_shape=(50, 37, 1)):
    """
    Simulate the first part of the VGG-Face model on the client side.
    """
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    outputs = tf.keras.layers.Flatten()(x)
    return tf.keras.Model(inputs, outputs, name="Client_Model")

def create_server_model(input_shape=(592,)):
    """
    Simulate the remaining part of the VGG-Face model on the server side.
    """
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(len(lfw.target_names), activation='softmax')(x)
    return tf.keras.Model(inputs, outputs, name="Server_Model")

# Step 4: Combine Models and Add Privacy
client_model = create_client_model(input_shape=(50, 37, 1))
server_model = create_server_model(input_shape=(client_model.output_shape[1],))

# Step 5: Batch Training with Simulated Partitioning
# Prepare data
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension
X_test = np.expand_dims(X_test, axis=-1)

batch_size = 32

def privacy_preserving_training(client_model, server_model, X_train, y_train, X_test, y_test, epsilon=2.0, epochs=3):
    """
    Simulate training with privacy-preserving data transfer using batches.
    """
    # Compile the server model
    server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        # Shuffle and process data in batches
        idx = np.random.permutation(len(X_train))
        X_train_shuffled, y_train_shuffled = X_train[idx], y_train[idx]

        for start in range(0, len(X_train), batch_size):
            end = start + batch_size
            X_batch, y_batch = X_train_shuffled[start:end], y_train_shuffled[start:end]

            # Client-side processing
            client_output = client_model.predict(X_batch)
            noisy_output = add_gaussian_noise(client_output, epsilon).numpy()

            # Server-side processing
            server_model.train_on_batch(noisy_output, y_batch)

        # Evaluate after each epoch
        print("Evaluating...")
        client_output_test = client_model.predict(X_test)
        noisy_output_test = add_gaussian_noise(client_output_test, epsilon).numpy()
        test_loss, test_accuracy = server_model.evaluate(noisy_output_test, y_test, verbose=0)
        print(f"Test Accuracy after epoch {epoch+1}: {test_accuracy * 100:.2f}%")

# Train the model
privacy_preserving_training(client_model, server_model, X_train, y_train, X_test, y_test, epsilon=2.0, epochs=3)


Loading Labeled Faces in the Wild dataset...
Dataset loaded: 1404 training samples, 156 testing samples

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━